In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
# Initialize Spark Session
spark = SparkSession.builder.appName("Stores").getOrCreate()

# Define Data
data = [
    ('S1', 'Q1', 200), ('S1', 'Q2', 300), ('S1', 'Q4', 400),
    ('S2', 'Q1', 500), ('S2', 'Q3', 600), ('S2', 'Q4', 700),
    ('S3', 'Q1', 800), ('S3', 'Q2', 750), ('S3', 'Q3', 900)
]

columns = ["Store", "Quarter", "Amount"]

# Create DataFrame
stores_df = spark.createDataFrame(data, columns)

# Show Data
stores_df.show()
stores_df.createOrReplaceTempView("stores")


+-----+-------+------+
|Store|Quarter|Amount|
+-----+-------+------+
|   S1|     Q1|   200|
|   S1|     Q2|   300|
|   S1|     Q4|   400|
|   S2|     Q1|   500|
|   S2|     Q3|   600|
|   S2|     Q4|   700|
|   S3|     Q1|   800|
|   S3|     Q2|   750|
|   S3|     Q3|   900|
+-----+-------+------+



In [12]:
spark.sql(
"""
    select store, 10 - sum(cast(right(Quarter, 1) as int)) from stores
    group by store
""").show()

+-----+------------------------------------------+
|store|(10 - sum(CAST(right(Quarter, 1) AS INT)))|
+-----+------------------------------------------+
|   S1|                                         3|
|   S2|                                         2|
|   S3|                                         4|
+-----+------------------------------------------+



In [15]:
spark.sql("""
    with cte as (
        select distinct store, 1 as q_no from stores
        union all
        select store, q_no + 1 as q_no from cte
        where q_no < 4
    )
    select * from cte
    """
)

AnalysisException: Table or view not found: cte; line 5 pos 44;
'WithCTE
:- 'CTERelationDef 0, false
:  +- 'SubqueryAlias cte
:     +- 'Union false, false
:        :- Distinct
:        :  +- Project [store#34, 1 AS q_no#98]
:        :     +- SubqueryAlias stores
:        :        +- View (`stores`, [Store#34,Quarter#35,Amount#36L])
:        :           +- LogicalRDD [Store#34, Quarter#35, Amount#36L], false
:        +- 'Project ['store, ('q_no + 1) AS q_no#99]
:           +- 'Filter ('q_no < 4)
:              +- 'UnresolvedRelation [cte], [], false
+- 'Project [*]
   +- 'SubqueryAlias cte
      +- 'CTERelationRef 0, false
